# Step 1)
Download the postgresql Chembl_24 database from here https://www.ebi.ac.uk/chembl/downloads

The total size of the database is ~1.2 GB so it may take some time. Once downloaded, you can restore the database by following the instructions on included with the tarfile. This will take >30 minutes.

# Step 2)

We use psycopg2, a python-based postgresql adapter to communicate with the database and extract the information we are interested in.

Unfortunately, the Chembl_24 is very complex. You can take a look at the schema here ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_24_1_schema.png

For this tutorial, we will retrieve all the molecules with a pchembl affinity of > 2 associated with the 107 proteins below

The syntax for the retrieval is normal sql, you can take a look at the schema documentation for each table here ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_24_1_schema_documentation.html

In [ ]:
import psycopg2
conn = psycopg2.connect(dbname='XXXX', user='XXXX', host='XXXX', password='XXXX')   ## connect to the database
curr = conn.cursor() 

From the CHEMBL schema, we are interested in the following items,

a) the CHEMBL_id of the molecules

b) the MOLREGNO id

c) the preferred name

d) canonical smiles

e) and standard INCHI data

By grabbing all of these information, we can go back to the CHEMBL database to get proteins of interests

In [ ]:
curr.execute("SELECT md.chembl_id, md.MOLREGNO, md.PREF_NAME, cs.canonical_smiles, cs.STANDARD_INCHI, cs.STANDARD_INCHI_KEY \
            FROM molecule_dictionary md JOIN compound_structures cs ON md.molregno = cs.molregno")     

In [ ]:
result_data = curr.fetchall()    ##run execute to fetch data

In [ ]:
curr.close()    ## dont forget to close your connection!

# Step 3

Once we have the list of molecules of interest, we can use RDKIT, a very popular chemoinformatics package in python, to fragment molecules and calculate molecular properties

In the following sections, we used the BRICS Fragmentation Scheme to fragment molecules into constituent fragments and save them into a json file

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import BRICS

In the following sections, 

a) we process the molecules in blocks of 50,000 molecules, molecules with more than 100 heavy atoms are filtered out as these usually are peptides or complex carbohydrates. 

b) we apply BRICS fragmentation using the BRICS.BRICSDecompose command and save the fragments into a list. '[Xe]' is used as a placeholder for where BRICS performs the fragmentation. 

c) Finally, the molecules are saved into json files for easy reading/loading into elasticsearch

!!!! VERY IMPORTANT !!!!
BRICS Fragmentation is not perfect! For larger cyclic molecules, BRICS may accidentally get stuck into a recursive loop trying to fragment the molecule. In this case, theres no easy escape other than a hard interrupution of the kernel in jupyter notebook. Be warned that this may happen!


In [ ]:
for a,b in [(100000,150000)]:
    data_dictionary_list = []
    non_fragments = []
    for dx,dat in enumerate(result_data[a:b]):
        temp_dict={'CHEMBL_ID':dat[0],'MOL_REGNO':dat[1],'SMILES':dat[3],'STANDARD_INCHI':dat[4]}
        try:
            m = Chem.MolFromSmiles(dat[3])
            if m.GetNumHeavyAtoms() < 100:
                frags = list(BRICS.BRICSDecompose(m))
                frag_list = []
                for frag in frags:
                    frag_sub = dummy_re.sub('[Xe]',frag)
                    if '[Xe]' in frag_sub:
                        frag_list.append(frag_sub)
                if frag_list:
                    temp_dict['FRAGMENTS'] = frag_list
                    data_dictionary_list.append(temp_dict)
        except:
            non_fragments.append((dx,dat[0]))
        if dx%1000==0:
                print(a+dx)
    with open('CHEMBL_Molecules_fragments_'+str(a)+'_'+str(b)+'.json','w') as f:
        json.dump(data_dictionary_list,f)

RDKIT is more powerful than just BRICS, one can also calculate molecular properties like

a) Fingerprints

b) Gasteiger Charges

c) 3D Conformations

d) Molecular Features

e) and much more!

Hopefully, this will be an introduction for you to RDKIT!